In [1]:
import lightgbm as lgb
from factory import create_data_loader

In [2]:
from evaluation.generalevaluator import Evaluator
import random

In [6]:
data_loader = create_data_loader("iris", test_size=0.2, normalize_features = "mean_std", return_extra_info = True, encode_categorical=True)
X_train, X_val, y_train, y_val, extra_info = data_loader.load_data()

In [7]:
param_grid = {  
    'objective' : ["multiclass"],
    'num_classes' : [3],
    'num_leaves': [16, 31],
    'max_depth': [2, 12,-1],
    'max_bin': [4],
    'learning_rate': [0.1, 0.2],
    'n_estimators': [2,3],

    'min_child_weight': [0.001, 0.01],
    'min_child_samples': [20, 50],
    'subsample': [0.8, 1.0],
    'reg_alpha': [0.0, 5.0],
    'reg_lambda': [0.0, 5.0]}

params = {k:random.choice(v) for k,v in param_grid.items()}

In [12]:
model = lgb.LGBMRegressor(num_iterations = 120)

In [13]:
model.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000040 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 87
[LightGBM] [Info] Number of data points in the train set: 120, number of used features: 4
[LightGBM] [Info] Start training from score 1.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

LGBMRegressor(num_iterations=120)

In [182]:
params

{'objective': 'multiclass',
 'num_classes': 3,
 'num_leaves': 16,
 'max_depth': 2,
 'max_bin': 4,
 'learning_rate': 0.2,
 'n_estimators': 3,
 'min_child_weight': 0.001,
 'min_child_samples': 20,
 'subsample': 1.0,
 'colsample_bytree': 0.8,
 'reg_alpha': 5.0,
 'reg_lambda': 0.0}

In [9]:
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

In [184]:
model = lgb.train(params, train_data, valid_sets=[val_data])


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000467 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16
[LightGBM] [Info] Number of data points in the train set: 120, number of used features: 4
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn

In [185]:
y_pred = model.predict(X_val.values)
#y_pred = (y_pred>0.5).astype(int)
evaluator = Evaluator(
                y_true=y_val,
                y_pred=y_pred,
                run_metrics=["accuracy",],
                metric="mse",
                problem_type="multiclass_classification",
            )
output_metrics = evaluator.evaluate_model()
print(output_metrics)

{'accuracy': 0.3333333333333333}


In [186]:
y_pred

array([[0.51045954, 0.29937546, 0.190165  ],
       [0.20546507, 0.54263084, 0.25190409],
       [0.51045954, 0.29937546, 0.190165  ],
       [0.51045954, 0.29937546, 0.190165  ],
       [0.55218679, 0.24210328, 0.20570994],
       [0.22583811, 0.49728007, 0.27688182],
       [0.55218679, 0.24210328, 0.20570994],
       [0.55218679, 0.24210328, 0.20570994],
       [0.1853079 , 0.23080857, 0.58388353],
       [0.21250416, 0.561221  , 0.22627484],
       [0.21796772, 0.2714877 , 0.51054458],
       [0.18373741, 0.23732752, 0.57893507],
       [0.1853079 , 0.23080857, 0.58388353],
       [0.22583811, 0.49728007, 0.27688182],
       [0.55218679, 0.24210328, 0.20570994],
       [0.55218679, 0.24210328, 0.20570994],
       [0.55218679, 0.24210328, 0.20570994],
       [0.21250416, 0.561221  , 0.22627484],
       [0.20546507, 0.54263084, 0.25190409],
       [0.20546507, 0.54263084, 0.25190409],
       [0.55218679, 0.24210328, 0.20570994],
       [0.18373741, 0.23732752, 0.57893507],
       [0.

In [180]:
class_predictions = np.argmax(y_pred, axis=1)
class_predictions

array([0, 2, 0, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 2, 0, 2,
       1, 1, 2, 2, 1, 0, 2, 0])

In [164]:
import numpy as np

# Assuming y_pred is the array you provided
# Convert intermediate outputs to class predictions


# Define a list of class labels (replace with your actual class labels)
class_labels = ['Class 0', 'Class 1', 'Class 2']

# Map indices to class labels
predicted_classes = [class_labels[prediction] for prediction in class_predictions]

print("Predicted classes:", predicted_classes)

AxisError: axis 1 is out of bounds for array of dimension 1